In [22]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
from shapely.geometry import LineString, Point, Polygon, MultiPolygon , GeometryCollection
from shapely.ops import unary_union

import pyproj
from pyproj import Transformer

import numpy as np
import matplotlib.pyplot as plt
import fiona
import rasterio
from rasterio.plot import show, show_hist
from rasterio.merge import merge
from rasterio.windows import WindowError

import PIL
from PIL import Image, ImageTk, ImageDraw

from rasterio.transform import Affine
from rasterio.mask import mask
from rasterio.crs import CRS  # Import CRS from rasterio.crs
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from scipy.ndimage import zoom

import xml.etree.ElementTree as ET

import zipfile

import os 

idx = pd.IndexSlice



In [23]:
upath = r'../data'

In [24]:
def extract_specific_rooftype_height_and_geometry(gml_file, specific_rooftype):
    # Parse the CityGML file
    tree = ET.parse(gml_file)
    root = tree.getroot()

    # Namespace mapping for CityGML
    ns = {
        'citygml': 'http://www.opengis.net/citygml/1.0',
        'building': 'http://www.opengis.net/citygml/building/1.0',
        'gml': 'http://www.opengis.net/gml'
    }

    # List to store buildings with the specific rooftop type
    buildings_info = []

    # Find all buildings
    for building in root.findall('.//building:Building', ns):
        roof_type_elem = building.find('.//building:roofType', ns)
        
        if roof_type_elem is not None and roof_type_elem.text.lower() == specific_rooftype.lower():
            object_id = building.get('{http://www.opengis.net/gml}id', 'Unknown ID')
            
            height_elem = building.find('.//building:measuredHeight', ns)
            height = float(height_elem.text) if height_elem is not None else 'Unknown height'
            
            # Extract geometry
            geometrylist = []
            for polygon_elem in building.findall('.//gml:Polygon', ns):
                exterior_elem = polygon_elem.find('gml:exterior/gml:LinearRing', namespaces=ns)
                if exterior_elem is not None:
                    pos_list_elem = exterior_elem.find('gml:posList', namespaces=ns)
                    if pos_list_elem is not None:
                        coords_str = pos_list_elem.text.strip()
                        coords_list = coords_str.split()
                        coords = []
                        for i in range(0, len(coords_list), 3):  # Assuming 3D coordinates (x, y, z)
                            x = float(coords_list[i])
                            y = float(coords_list[i+1])
                            z = float(coords_list[i+2])
                            coords.append((x, y, z))
                        polygon_geom = Polygon(coords)
                        if not polygon_geom.is_valid:
                            polygon_geom = polygon_geom.buffer(0)  # Fix invalid geometries
                        if polygon_geom.is_valid:
                            geometrylist.append(polygon_geom)
                        
                        
            if geometrylist:
                geometry = unary_union(geometrylist)
            else:
                geometry = None

            buildings_info.append({
                'object_id': object_id,
                'height': height,
                'geometry': geometry
            })

    # Create a DataFrame from the extracted information
    df = gpd.GeoDataFrame(buildings_info)

    return df

In [25]:
def dissolve_polygons(buildings_3d):
    data_list = []

    # Iterate through each row in buildings_3d
    for index, row in buildings_3d.iterrows():
        all_parts = []

        # Iterate through each polygon geometry in the row
        for geom in row['geometry']:
            # Convert 3D to 2D
            exterior_coords = list(geom.exterior.coords.xy)
            polygon_part = Polygon(zip(exterior_coords[0], exterior_coords[1]))
            all_parts.append(polygon_part)

        # Create a GeometryCollection from all parts
        geom_collection = GeometryCollection(all_parts)

        try:
            # Union all parts to create a single Polygon
            dissolved_polygon = unary_union(geom_collection)
        except Exception as e:
            print(f"Error during union operation: {e}")
            continue  # Skip this geometry if union fails

        # Append dissolved polygon and object_id to data_list
        data_list.append({
            'geometry': dissolved_polygon,
            'object_id': row['object_id']  # Assuming 'object_id' is a column in buildings_3d
        })

    # Create a GeoDataFrame from the data list
    gdf = gpd.GeoDataFrame(data_list, geometry='geometry')

    return gdf

In [26]:
%%time
# Extract information for buildings with specific rooftop type (e.g., "satteldach")
specific_rooftype = "SATTELDACH"

# List to store paths .gml as well as 3D and 2D polygon  files
df_buildings_3d = gpd.GeoDataFrame()
df_buildings_2d = gpd.GeoDataFrame()

dir_path = os.path.join(upath, r'Vienna/LOD2.1/unzip')

# Iterate over files in the directory
for fn in os.listdir(dir_path):
    file_path = os.path.join(dir_path, fn)
    if os.path.isfile(file_path) and fn.lower().endswith('.gml'):
        #print(fn)
        
        # Extract specific rooftype height and geometry
        buildings_3d = extract_specific_rooftype_height_and_geometry(file_path, specific_rooftype)
        print(buildings_3d.head())
        #buildings_3d.reset_index(drop=True).to_csv(os.path.join(upath, r'Vienna/modelling_results/building_3d_TEST.csv'))
        if buildings_3d is not None and not buildings_3d.empty:
            # Concatenate the 3D buildings
            df_buildings_3d = pd.concat([df_buildings_3d, buildings_3d])

            # Convert 3D buildings to 2D and concatenate
            # try:
            #     buildings_2d = dissolve_polygons(buildings_3d)
            #     df_buildings_2d = pd.concat([df_buildings_2d, buildings_2d])
            # except ValueError as e:
            #     print(f"Error converting {fn} to 2D: {e}")
        else:
            print(f"No data for specific rooftype in file {fn}")
            
# Save to file            
df_buildings_3d.reset_index(drop=True).to_csv(os.path.join(upath, r'Vienna/modelling_results/building_3d_2_1.csv'))
#df_buildings_2d.reset_index(drop=True).to_file(os.path.join(upath, r'Vienna/modelling_results/building_2d_2.json'), driver='GeoJSON')

                                  object_id  height   
0  UUID_LOD2_034769-15982479-db49-4af1-b7a3    4.18  \

                                            geometry  
0  POLYGON Z ((-10281.051 336476.089 239.507, -10...  
                                  object_id  height   
0  UUID_LOD2_043498-5ce690b4-7be3-470a-8084    8.49  \
1  UUID_LOD2_092812-338cbe26-e65c-4bde-bcbd    9.21   
2  UUID_LOD2_097704-593799f2-a8cb-4de7-8e74    7.31   
3  UUID_LOD2_103166-6c75acd8-c5b0-465c-8bdf    6.02   
4  UUID_LOD2_103166-de1edcf4-4449-4f70-995a    8.58   

                                            geometry  
0  POLYGON Z ((-10198.134 342927.052 270.66, -101...  
1  POLYGON Z ((-10280.519 342900 260.79, -10281.2...  
2  POLYGON Z ((-10402.482 342889.534 269.4, -1040...  
3  POLYGON Z ((-10212.579 342807.396 292.41, -102...  
4  POLYGON Z ((-10186.772 342864.624 281.6, -1018...  
                                  object_id  height   
0  UUID_LOD2_043488-5d480ad9-197f-4984-9776    7.00  \
1  UUID_